# Running on ServiceX

Some short examples at this point.

If you disconnect from the internet in the middle of a query execution, it will continue to execute inside ServiceX. You can then re-run the query, and the software should realize it was interupted, and fetch the results from the query you already made.

In [ ]:
from func_adl import ObjectStream
from config import ds_xaod

## UpROOT

Here is a very simple query based on the uproot backend, using ServiceX as the target:

In [2]:
import pandas as pd
from func_adl_servicex import ServiceXSourceUpROOT

dataset_name = "data15_13TeV:data15_13TeV.00282784.physics_Main.deriv.DAOD_PHYSLITE.r13286_p4910_p5226"
dataset_name = "user.kchoi:user.kchoi.ttHML_80fb_ttH"
# src = ServiceXSourceUpROOT(dataset_name, "CollectionTree")
# data = src.Select("lambda e: {'JetPT': e['AnalysisJetsAuxDyn.pt']}") \
#     .AsParquetFiles('junk.parquet') \
#     .value()
src = ServiceXSourceUpROOT(dataset_name, "nominal")
data = src.Select("lambda e: {'M101': e['Mlll012']}") \
    .AsParquetFiles('junk.parquet') \
    .value()
df = pd.read_parquet(data[0])
print(df)

user.kchoi:user.kcho...:   0%|          | 0/9000000000.0 [00:00]

        user.kchoi:user.kcho... Downloaded:   0%|          | 0/9000000000.0 [00:00]

                 M101
0            0.000000
1       197345.343750
2       205475.250000
3            0.000000
4       136593.234375
...               ...
257059  303052.437500
257060       0.000000
257061  133332.531250
257062       0.000000
257063  110212.789062

[257064 rows x 1 columns]


In [9]:
from func_adl_servicex import ServiceXSourceUpROOT
dataset_name = ["https://xrootd-local.unl.edu:1094//store/user/AGC/nanoAOD/nanoaod15.root"]
import servicex
sx_dataset = servicex.ServiceXDataset(dataset_name, "uproot")
ds = ServiceXSourceUpROOT(sx_dataset, "Events")
data = ds.value()
assert len(data) == 1
import awkward as ak
d = ak.from_parquet(data[0])
d

<Array [{run: 1, ... ] type='349 * {"run": uint32, "luminosityBlock": uint32, "e...'>

## ATLAS R21 XAOD

## CMS Run 1 AOD

## Common Features

There are a few things that are common to running on the `ServiceX` backend. Most, but not all, of these are a function of the front-end library. When using `func_adl` direct access to this library is mostly hidden.

* [Introduction to the `servicex` library and the up-to-date details.](https://github.com/ssl-hep/ServiceX_frontend)
* [Releases](https://github.com/ssl-hep/ServiceX_frontend/releases) of the library, which should give you an idea of what has changed most recently.
* See the [chapter on the frontend `servicex` library's features](ch-controlling-sx).

## `ServiceX` Dashboard

Lets start with the dashboard, which is not part of the front-end library. Rather, it is one of the pages served from the service itself. Here is a typical picture while a query is running: 

## Query Title

It is possible to add a title to your query. The title will appear on the `ServiceX` dashboard (the dash board is very common) - and is used no where else. This simply makes life easier if your code ends up running 20 or 30 queries to tell each one apart. The query is added to the `title` parameter of `value` method:

In [ ]:
help(ObjectStream.value)

Note that the `value` and `value_async` have the same signature. See the [`async` ServiceX](sec-async) section for more information about the `async` API.

Here is what the actual query looks like and how it shows up on the ServiceX Dashboard:

In [ ]:
v_jets = (ds_xaod
          .Select(lambda e: e.Jets())
          .Select(lambda jets: [j for j in jets if j.pt()/1000.0 > 30])
          .Select(lambda jets: {
              'pt': [j.pt()/1000.0 for j in jets],
          })
          .AsAwkwardArray()
          .value(title="Test Job")
)

While it is running the job name will be included in the marching ant's display:

```{image} assets/notebook_title_job_running_status.png
:alt: Notebook Marching Ants
:width: 600px
:align: center
```

And the dash board will also be labeled:

```{image} assets/dashboard_with_title.png
:alt: Dashboard with transform title
:width: 600px
:align: center
```

Unless you are on an open unauthenticated ServiceX instance, only you will be able to see the title (along with the sys-admins!).